In [1]:
data_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/"

In [2]:
import pandas as pd

def retrieve_test_data(data_path):
    test_df = pd.read_csv(data_path + 'test_series_descriptions.csv')

    return test_df

retrieve_test_data(data_path)

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [3]:
import os

def retrieve_image_paths(base_path, study_id, series_id):
    series_dir = os.path.join(base_path, str(study_id), str(series_id))
    images = os.listdir(series_dir)
    image_paths = [os.path.join(series_dir, img) for img in images]
    return image_paths

In [4]:
import pydicom

def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [5]:
import numpy as np
from enum import Enum
from torch.utils.data import Dataset, DataLoader

CONDITIONS = {
    "Sagittal T2/STIR": ["Spinal Canal Stenosis"],
    "Axial T2": ["Left Subarticular Stenosis", "Right Subarticular Stenosis"],
    "Sagittal T1": ["Left Neural Foraminal Narrowing", "Right Neural Foraminal Narrowing"],
}
MAX_IMAGES_IN_SERIES = {
    "Sagittal T2/STIR": 29,
    "Axial T2": 192,
    "Sagittal T1": 38,
}


class SeriesDataType(Enum):
    SEQUENTIAL_VARIABLE_LENGTH = 1
    SEQUENTIAL_VARIABLE_LENGTH_WITH_CLS = 2
    SEQUENTIAL_FIXED_LENGTH = 3
    CUBE_3D = 4


class SeriesLevelTestset(Dataset):
    def __init__(self,
                 base_path: str,
                 dataframe: pd.DataFrame,
                 data_type=SeriesDataType.SEQUENTIAL_FIXED_LENGTH,
                 data_series="Sagittal T2/STIR",
                 transform=None):
        self.base_path = base_path
        self.type = data_type
        self.data_series = data_series

        self.dataframe = (dataframe[['study_id', "series_id"]]
                          .drop_duplicates())
        self.series = self.dataframe[['study_id', "series_id"]].drop_duplicates().reset_index(drop=True)

        self.transform = transform


    def __len__(self):
        return len(self.series)

    def __getitem__(self, index):
        curr = self.series.iloc[index]
        image_paths = retrieve_image_paths(self.base_path, curr["study_id"], curr["series_id"])
        image_paths = sorted(image_paths, key=lambda x: self._get_image_index(x))

        images = np.array([np.array(self.transform(image=load_dicom(image_path))['image']) if self.transform
                           else load_dicom(image_path) for image_path in image_paths])

        if self.type == SeriesDataType.SEQUENTIAL_FIXED_LENGTH:
            front_buffer = (MAX_IMAGES_IN_SERIES[self.data_series] - len(images)) // 2
            rear_buffer = (MAX_IMAGES_IN_SERIES[self.data_series] - len(images)) // 2 + (
                        (MAX_IMAGES_IN_SERIES[self.data_series] - len(images)) % 2)
            if len(images) <= MAX_IMAGES_IN_SERIES[self.data_series]:
                images = np.pad(images, ((front_buffer, rear_buffer), (0, 0), (0, 0)))
            else:
                diff = len(images) - MAX_IMAGES_IN_SERIES[self.data_series]
                images = images[diff // 2: diff // 2 + MAX_IMAGES_IN_SERIES[self.data_series]]
            
        elif self.type == SeriesDataType.SEQUENTIAL_VARIABLE_LENGTH_WITH_CLS:
            images = np.pad(images, ((1, 0), (0, 0), (0, 0)))

        elif self.type == SeriesDataType.CUBE_3D:
            width = len(images[0])
            images = np.pad(images, ((0, width - len(images)), (0, 0), (0, 0)))

        return images, curr["study_id"]


    def _get_image_index(self, image_path):
        return int(image_path.split("/")[-1].split("\\")[-1].replace(".dcm", ""))

In [6]:
def create_series_level_testset_and_loader(df: pd.DataFrame,
                                             series_description: str,
                                             transform,
                                             base_path: str,
                                             batch_size=1,
                                             num_workers=0,
                                             data_type=SeriesDataType.SEQUENTIAL_FIXED_LENGTH):
    test_df = df[df['series_description'] == series_description]
    
    testset = SeriesLevelTestset(base_path, test_df, transform=transform, data_type=data_type, data_series=series_description)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return testset, test_loader

In [7]:
import albumentations as A

transform = A.Compose([
    A.Resize(384,384),
    A.Normalize(mean=0.5, std=0.5),
])

In [8]:
data = retrieve_test_data(data_path)

testset_t1, test_loader_t1 = create_series_level_testset_and_loader(data, "Sagittal T1", transform, 
                                                                    os.path.join(data_path, "test_images"))
testset_t2, test_loader_t2 = create_series_level_testset_and_loader(data, "Axial T2", transform, 
                                                                    os.path.join(data_path, "test_images"))
testset_t2stir, test_loader_t2stir = create_series_level_testset_and_loader(data, "Sagittal T2/STIR", transform, 
                                                                    os.path.join(data_path, "test_images"))

In [9]:
import torch.nn as nn
import timm

class CNN_Model_Multichannel(nn.Module):
    def __init__(self, backbone="tf_efficientnetv2_b3", in_chans=29, num_levels=5, pretrained=False):
        super(CNN_Model_Multichannel, self).__init__()

        self.num_levels = num_levels
        self.encoder = timm.create_model(
            backbone,
            num_classes=CONFIG["out_dim"] * self.num_levels,
            features_only=False,
            drop_rate=CONFIG["drop_rate"],
            drop_path_rate=CONFIG["drop_path_rate"],
            pretrained=pretrained,
            # !TODO: Refactor
            in_chans=CONFIG["in_chans"] * in_chans,
        )

    def forward(self, x):
        return self.encoder(x).reshape((-1, self.num_levels, 3))

In [10]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [11]:
t1_model = torch.load("/kaggle/input/rsna-2024/pytorch/efficientnet_b4_multichannel/1/efficientnet_b4_multichannel_t1_95.pt").to(device)
t2_model = torch.load("/kaggle/input/rsna-2024/pytorch/efficientnet_b4_multichannel/1/efficientnet_b4_multichannel_t2_85.pt").to(device)

In [12]:
CONDITIONS = {
    "Sagittal T2/STIR": ["spinal_canal_stenosis"],
    "Axial T2": ["left_subarticular_stenosis", "right_subarticular_stenosis"],
    "Sagittal T1": ["left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing"],
}

LEVELS = ["l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"]

results_df = pd.DataFrame({"row_id":[], "normal_mild": [], "moderate": [], "severe": []})

In [13]:
softmax = nn.Softmax(dim=2)

In [14]:
len(test_loader_t1), len(testset_t1)

(1, 1)

In [15]:
testset_t1.dataframe

,study_id,series_id
0,44036939,2828203845


In [16]:
with torch.no_grad():
    t1_model.eval()

    for images, study_id in test_loader_t1:
        output = t1_model(images.to(device))
        output = softmax(output)
        output = output.detach().cpu().numpy()[0]
        for index, level in enumerate(output):
            row_id = f"{study_id.item()}_{CONDITIONS['Sagittal T1'][index % 2]}_{LEVELS[index // 2]}"
            results_df = results_df._append({"row_id": row_id, "normal_mild": level[0], "moderate": level[1], "severe": level[2]}, ignore_index=True)

/tmp/ipykernel_23/3625421379.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = results_df._append({"row_id": row_id, "normal_mild": level[0], "moderate": level[1], "severe": level[2]}, ignore_index=True)


In [17]:
with torch.no_grad():
    t2_model.eval()

    for images, study_id in test_loader_t2:
        output = t2_model(images.to(device))
        output = softmax(output)
        output = output.detach().cpu().numpy()[0]
        for index, level in enumerate(output):
            row_id = f"{study_id.item()}_{CONDITIONS['Axial T2'][index % 2]}_{LEVELS[index // 2]}"
            results_df = results_df._append({"row_id": row_id, "normal_mild": level[0], "moderate": level[1], "severe": level[2]}, ignore_index=True)

In [18]:
class CNN_Model_Multichannel(nn.Module):
    def __init__(self, backbone="tf_efficientnetv2_b3", in_chans=29, num_levels=5, pretrained=False):
        super(CNN_Model_Multichannel, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            num_classes=CONFIG["out_dim"] * 5,
            features_only=False,
            drop_rate=CONFIG["drop_rate"],
            drop_path_rate=CONFIG["drop_path_rate"],
            pretrained=pretrained,
            # !TODO: Refactor
            in_chans=CONFIG["in_chans"] * in_chans,
        )

    def forward(self, x):
        return self.encoder(x).reshape((-1, 5, 3))

t2stir_model = torch.load("/kaggle/input/rsna-2024/pytorch/efficientnet_b4_multichannel/1/efficientnet_b4_multichannel_t2stir_100.pt").to(device)

In [19]:
with torch.no_grad():
    t2stir_model.eval()

    for images, study_id in test_loader_t2stir:
        output = t2stir_model(images.to(device))
        output = softmax(output)
        output = output.detach().cpu().numpy()[0]
        for index, level in enumerate(output):
            row_id = f"{study_id.item()}_{CONDITIONS['Sagittal T2/STIR'][0]}_{LEVELS[index]}"
            results_df = results_df._append({"row_id": row_id, "normal_mild": level[0], "moderate": level[1], "severe": level[2]}, ignore_index=True)

In [20]:
results_df

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.934461,0.060538,0.005001
1,44036939_right_neural_foraminal_narrowing_l1_l2,0.708167,0.271924,0.019909
2,44036939_left_neural_foraminal_narrowing_l2_l3,0.637843,0.359019,0.003138
3,44036939_right_neural_foraminal_narrowing_l2_l3,0.700618,0.298546,0.000836
4,44036939_left_neural_foraminal_narrowing_l3_l4,0.353530,0.634238,0.012231
5,44036939_right_neural_foraminal_narrowing_l3_l4,0.469407,0.528294,0.002298
6,44036939_left_neural_foraminal_narrowing_l4_l5,0.383200,0.525499,0.091301
7,44036939_right_neural_foraminal_narrowing_l4_l5,0.618387,0.345188,0.036426
8,44036939_left_neural_foraminal_narrowing_l5_s1,0.211193,0.359611,0.429197
9,44036939_right_neural_foraminal_narrowing_l5_s1,0.215729,0.073193,0.711077


In [21]:
results_df.to_csv('submission.csv', index=False)